<a href="https://colab.research.google.com/github/Xiezhihaa/QOSF-Screening-Task/blob/main/Task%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

QOSF Task2 Odd to Even

In [ ]:
!pip install qiskit
!pip install qiskit[visualization]
!pip install qiskit_aer

In [ ]:
!pip install --upgrade qiskit
!pip install --upgrade qiskit-aer

In [67]:
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT
import numpy as np
from numpy import pi
from qiskit.primitives import Sampler

In [8]:
input_list = [1,2,2,4,5,6,7,11,17,21,22,23]
n = 31

Classical computer

In [4]:
def odd_to_even(n,numbers):
    converted = []
    for num in numbers:
        if num%2 != 0:
            num += 1
        if num >= n:
            a = 0
            a = num - n
            num = num - a
            if num%2 != 0:
                num -= 1
        converted.append(num)
    return converted

print(odd_to_even(n,input_list))

[2, 2, 2, 4, 6, 6, 8, 12, 18, 22, 22, 24]


Quantum Computer

In [5]:
def initialize_qubits(circuit, qubits, number):
    """Initialize the qubits to a specific number in binary representation."""
    binary = bin(number)[2:].zfill(len(qubits))
    for i, bit in enumerate(reversed(binary)):
        if bit == '1':
            circuit.x(qubits[i])

In [134]:
def apply_qft(circuit, qubits):
  circuit.append(QFT(len(qubits), do_swaps=False),qubits)

# LSB is qubits[0], apply phase rotation to all qubits if LSB is 1 (odd number)
def conditional_increment(circuit,qubits):
  for i in range(1, len(qubits)):
    circuit.cp(pi/(2**(i-1)), qubits[i],qubits[:-1])

def apply_inverst_qft(circuit,qubits):
  circuit.append(QFT(len(qubits),do_swaps=False).inverse(),qubits)

def measure(circuit, qubits, classical_bits):
  circuit.measure(qubits, classical_bits)

In [131]:
def build_circuit(number, n):
  n_qubits = len(bin(n).replace("0b", ""))

  qr = QuantumRegister(n_qubits)
  cr = ClassicalRegister(n_qubits)
  qc = QuantumCircuit(qr,cr)

  # Initialize the qubits to represent 'number'
  initialize_qubits(qc,qr,number)
  # QFT
  apply_qft(qc,qr)
  conditional_increment(qc,qr)
  apply_inverst_qft(qc,qr)

  # measure
  measure(qc,qr,cr)

  return qc

In [139]:
def odd_to_even_qc(numbers, n):
    # Quantum circuit to convert odd numbers to even
    results = []

    for number in numbers:
        #Odd to even

        circuit = build_circuit(number, n)

        # Execute the circuit
        simulator = AerSimulator()
        circ = transpile(circuit, simulator)
        job = simulator.run(circ, shots = 1024)
        result = job.result()
        counts = result.get_counts()
        max_count = max(counts, key = counts.get)
        answer = int(max_count, 2)
        results.append(answer)

    return results

input_list = [1,2,2,4,5,6,7,11,17,21,22,23]
print(input_list)
print(odd_to_even(n,input_list))

[1, 2, 2, 4, 5, 6, 7, 11, 17, 21, 22, 23]
[2, 2, 2, 4, 6, 6, 8, 10, 10, 10, 10, 10]
